### Compare structural measures across datasets

#### Datasets
- NIMHANS
- QPN
- PPMI

#### Measures
- Cortical thickness (FS)
- Regional volumes (FS)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nibabel as nib
from nilearn import datasets, surface, plotting


In [ ]:
projects_dir = f"/home/nikhil/projects/Parkinsons/neuro_arch/analysis" 

IDP_dir = f"{projects_dir}/IDP"
figs_dir = f"{projects_dir}/figures/ADPD/poster/"

qpn_release = "Jan_2024"

nimhans_serb_agg_data_dir = f"{IDP_dir}/nimhans_serb/agg_dfs/"
nimhans_metal_agg_data_dir = f"{IDP_dir}/nimhans_metal/agg_dfs/"
qpn_agg_data_dir = f"{IDP_dir}/qpn/{qpn_release}/agg_dfs/"
ppmi_agg_data_dir = f"{IDP_dir}/ppmi/agg_dfs/"

CT_DKT_csv = "CT_DKT_df.csv"
bilateral_vol_csv = "bilateral_vol_ASEG_df.csv"
global_vol_csv = "global_vol_ASEG_df.csv"

demo_cols = ["participant_id","bids_id","age","sex","group","hemi","ds"]

In [ ]:
from enum import Enum

# Poster colors
class my_colors(Enum):
    NIM_SERB_CONTROL =  "#B5E48C"
    NIM_SERB_PD =       "#76C893"
    NIM_METAL_CONTROL = "#34A0A4"
    NIM_METAL_PD =      "#1E6091"

    QPN_CONTROL =       "#ffc971"
    QPN_PD =            "#ffb627"
    QPN_older_CONTROL = "#ff9505"
    QPN_older_PD =      "#e2711d"

    PPMI_CONTROL =      "#FFB3C1"
    PPMI_PD =           "#FF758F"
    PPMI_older_CONTROL= "#C9184A"
    PPMI_older_PD =     "#800F2F"
    

color_list = [  my_colors.NIM_SERB_CONTROL.value, my_colors.NIM_SERB_PD.value,
                my_colors.NIM_METAL_CONTROL.value, my_colors.NIM_METAL_PD.value,
                "white",
                my_colors.QPN_CONTROL.value, my_colors.QPN_PD.value, 
                my_colors.QPN_older_CONTROL.value, my_colors.QPN_older_PD.value,
                "white",
                my_colors.PPMI_CONTROL.value, my_colors.PPMI_PD.value,
                my_colors.PPMI_older_CONTROL.value, my_colors.PPMI_older_PD.value              
              ]

palette = sns.color_palette(palette=color_list)

hue_order = ["NIMHANS-1-control", "NIMHANS-1-PD", "NIMHANS-2-control", "NIMHANS-2-PD", "",
             "QPN-young-control", "QPN-young-PD", "QPN-older-control", "QPN-older-PD", "",
             "PPMI-young-control",  "PPMI-young-PD","PPMI-older-control",  "PPMI-older-PD"] 

sns.palplot(palette)

### Read data

In [ ]:
match_age = True
age_thresh = 63

In [ ]:
# NIMHANS (SERB + METAL)
nimhans_serb_CT_DKT_df = pd.read_csv(f"{nimhans_serb_agg_data_dir}{CT_DKT_csv}").drop(columns=["Unnamed: 0"])
nimhans_serb_CT_DKT_df["ds"] = "NIMHANS-1"
n_nimhans_serb_participants = len(nimhans_serb_CT_DKT_df["participant_id"].unique())

nimhans_metal_CT_DKT_df = pd.read_csv(f"{nimhans_metal_agg_data_dir}{CT_DKT_csv}").drop(columns=["Unnamed: 0"])
nimhans_metal_CT_DKT_df["ds"] = "NIMHANS-2"
n_nimhans_metal_participants = len(nimhans_metal_CT_DKT_df["participant_id"].unique())

# QPN
qpn_CT_DKT_df = pd.read_csv(f"{qpn_agg_data_dir}{CT_DKT_csv}").drop(columns=["Unnamed: 0"])
qpn_CT_DKT_df["ds"] = "QPN"
n_qpn_participants = len(qpn_CT_DKT_df["participant_id"].unique())

# PPMI
ppmi_CT_DKT_df = pd.read_csv(f"{ppmi_agg_data_dir}{CT_DKT_csv}").drop(columns=["Unnamed: 0"])
ppmi_CT_DKT_df["ds"] = "PPMI"
ppmi_CT_DKT_df["participant_id"] = ppmi_CT_DKT_df["participant_id"].astype(str)
ppmi_CT_DKT_df["bids_id"] = "sub-" + ppmi_CT_DKT_df["participant_id"]

n_ppmi_participants = len(ppmi_CT_DKT_df["participant_id"].unique())

print(f"n_nimhans_participants: {(n_nimhans_serb_participants, n_nimhans_metal_participants)}, n_qpn_participants (all):{n_qpn_participants}, n_ppmi_participants: {n_ppmi_participants}")

if match_age:
    print(f"Matching age < {age_thresh}")
    qpn_young_CT_DKT_df = qpn_CT_DKT_df[qpn_CT_DKT_df["age"] < age_thresh].copy()
    qpn_young_CT_DKT_df["ds"] = "QPN-young"

    qpn_older_CT_DKT_df = qpn_CT_DKT_df[qpn_CT_DKT_df["age"] >= age_thresh].copy()
    qpn_older_CT_DKT_df["ds"] = "QPN-older"

    ppmi_young_CT_DKT_df = ppmi_CT_DKT_df[ppmi_CT_DKT_df["age"] < age_thresh].copy()
    ppmi_young_CT_DKT_df["ds"] = "PPMI-young"

    ppmi_older_CT_DKT_df = ppmi_CT_DKT_df[ppmi_CT_DKT_df["age"] >= age_thresh].copy()
    ppmi_older_CT_DKT_df["ds"] = "PPMI-older"

    n_qpn_young_participants = len(qpn_young_CT_DKT_df["participant_id"].unique())
    n_qpn_older_participants = len(qpn_older_CT_DKT_df["participant_id"].unique())

    n_ppmi_young_participants = len(ppmi_young_CT_DKT_df["participant_id"].unique())
    n_ppmi_older_participants = len(ppmi_older_CT_DKT_df["participant_id"].unique())

    print(f"n_nimhans_participants: {(n_nimhans_serb_participants, n_nimhans_metal_participants)}")
    print(f"n_qpn_participants (young):{n_qpn_young_participants}, n_ppmi_participants (young): {n_ppmi_young_participants}")
    print(f"n_qpn_participants (older):{n_qpn_older_participants}, n_ppmi_participants: {n_ppmi_older_participants}")

# Concat
CT_DKT_df = pd.concat([nimhans_serb_CT_DKT_df, nimhans_metal_CT_DKT_df, 
                        qpn_young_CT_DKT_df, qpn_older_CT_DKT_df,
                        ppmi_young_CT_DKT_df, ppmi_older_CT_DKT_df], axis=0) 


CT_DKT_df["ds_group"] = CT_DKT_df["ds"] + "-" + CT_DKT_df["group"]
CT_DKT_df["ds_hemi"] = CT_DKT_df["ds"] + "\n" + CT_DKT_df["hemi"]
print(f"CT_DKT_df shape: {CT_DKT_df.shape}, n_total_participants: {len(CT_DKT_df['participant_id'].unique())}")


## tmp
# CT_DKT_df = CT_DKT_df.drop(columns=["participant_id"])

CT_DKT_df.head()

### Subsample QPN to match age

In [ ]:
CT_DKT_df.groupby(["ds","group","hemi"])["age"].describe()

In [ ]:
save_fig = True

# CT_DKT_df = CT_DKT_df[CT_DKT_df["dataset"].isin(["PPMI", "QPN", "NIMHANS_SERB"])]
CT_DKT_df = CT_DKT_df[CT_DKT_df["group"].isin(["control", "PD"])]

hue_order = ["NIMHANS-1-control", "NIMHANS-1-PD", "NIMHANS-2-control", "NIMHANS-2-PD", 
             "QPN-young-control", "QPN-young-PD", 
             "PPMI-young-control",  "PPMI-young-PD"]

CT_DKT_df_melt = CT_DKT_df.melt(
    id_vars=demo_cols + ["ds_group","ds_hemi"],
    var_name="ROI", 
    value_name="CTh (mm)")

plot_df = CT_DKT_df_melt.copy()
plot_df["ROI"] = plot_df["ROI"].astype(str)

sns.set_theme(font_scale=3)
with sns.axes_style("whitegrid"):
    g = sns.catplot(y="ROI",x="CTh (mm)", hue="ds_group", col="hemi",kind="point",palette=palette, hue_order=hue_order, legend=True,
                    data=plot_df, aspect=0.5, height=20)
    # g.tick_params(axis='x', rotation=90, labelsize=14)

if save_fig:
    g.savefig(f"{figs_dir}/DKT_point.png")

In [ ]:
# save_fig = False

# # CT_DKT_df = CT_DKT_df[CT_DKT_df["dataset"].isin(["PPMI", "QPN", "NIMHANS_SERB"])]
# CT_DKT_df = CT_DKT_df[CT_DKT_df["group"].isin(["control", "PD"])]

# color_list = [  "#969696", "#e7298a" ]

# palette_group = sns.color_palette(palette=color_list)

# CT_DKT_df_melt = CT_DKT_df.melt(
#     id_vars=demo_cols + ["ds_group","ds_hemi"],
#     var_name="ROI", 
#     value_name="CTh (mm)")

# plot_df = CT_DKT_df_melt.copy()
# plot_df["ROI"] = plot_df["ROI"].astype(str)

# sns.set_theme(font_scale=4)
# with sns.axes_style("whitegrid"):
#     g = sns.catplot(y="ROI",x="CTh (mm)", hue="group", col="ds_hemi", kind="box",palette=palette_group, legend=True,
#                     data=plot_df, aspect=0.2, height=30)
#     # g.tick_params(axis='x', rotation=90, labelsize=14)

# if save_fig:
#     g.savefig(f"{figs_dir}/DKT.png")

### Aseg bilateral volume

In [ ]:
# NIMHANS (SERB + METAL)
nimhans_serb_hemi_ASEG_df = pd.read_csv(f"{nimhans_serb_agg_data_dir}{bilateral_vol_csv}").drop(columns=["Unnamed: 0"])
nimhans_serb_hemi_ASEG_df["ds"] = "NIMHANS-1"
n_nimhans_serb_participants = len(nimhans_serb_hemi_ASEG_df["participant_id"].unique())

nimhans_metal_hemi_ASEG_df = pd.read_csv(f"{nimhans_metal_agg_data_dir}{bilateral_vol_csv}").drop(columns=["Unnamed: 0"])
nimhans_metal_hemi_ASEG_df["ds"] = "NIMHANS-2"
n_nimhans_metal_participants = len(nimhans_metal_hemi_ASEG_df["participant_id"].unique())

# QPN
qpn_hemi_ASEG_df = pd.read_csv(f"{qpn_agg_data_dir}{bilateral_vol_csv}").drop(columns=["Unnamed: 0"])
qpn_hemi_ASEG_df["ds"] = "QPN"
n_qpn_participants = len(qpn_hemi_ASEG_df["participant_id"].unique())

# PPMI
ppmi_hemi_ASEG_df = pd.read_csv(f"{ppmi_agg_data_dir}{bilateral_vol_csv}").drop(columns=["Unnamed: 0"])
ppmi_hemi_ASEG_df["ds"] = "PPMI"
n_ppmi_participants = len(ppmi_hemi_ASEG_df["participant_id"].unique())

print(f"n_nimhans_participants: {(n_nimhans_serb_participants,n_nimhans_metal_participants)}, n_qpn_participants:{n_qpn_participants}")

if match_age:
    print(f"Matching age < {age_thresh}")

    ppmi_younger_hemi_ASEG_df = ppmi_hemi_ASEG_df[ppmi_hemi_ASEG_df["age"] < age_thresh].copy()
    ppmi_younger_hemi_ASEG_df["ds"] = "PPMI-young"
    n_younger_ppmi_participants = len(ppmi_younger_hemi_ASEG_df["participant_id"].unique())

    ppmi_older_hemi_ASEG_df = ppmi_hemi_ASEG_df[ppmi_hemi_ASEG_df["age"] >= age_thresh].copy()
    ppmi_older_hemi_ASEG_df["ds"] = "PPMI-older"
    n_older_ppmi_participants = len(ppmi_older_hemi_ASEG_df["participant_id"].unique())

    qpn_younger_hemi_ASEG_df = qpn_hemi_ASEG_df[qpn_hemi_ASEG_df["age"] < age_thresh].copy()
    qpn_younger_hemi_ASEG_df["ds"] = "QPN-young"
    n_qpn_younger_participants = len(qpn_younger_hemi_ASEG_df["participant_id"].unique())

    qpn_older_hemi_ASEG_df = qpn_hemi_ASEG_df[qpn_hemi_ASEG_df["age"] >= age_thresh].copy()
    qpn_older_hemi_ASEG_df["ds"] = "QPN-older"
    n_qpn_older_participants = len(qpn_older_hemi_ASEG_df["participant_id"].unique())

    print(f"n_nimhans_participants: {(n_nimhans_serb_participants,n_nimhans_metal_participants)}")
    print(f"n_qpn_younger_participants:{n_qpn_younger_participants}, n_qpn_older_participants: {n_qpn_older_participants}")
    print(f"n_ppmi_younger_participants:{n_younger_ppmi_participants}, n_ppmi_older_participants: {n_older_ppmi_participants}")

# Concat
hemi_ASEG_df = pd.concat([nimhans_serb_hemi_ASEG_df, nimhans_metal_hemi_ASEG_df, 
                          qpn_younger_hemi_ASEG_df, qpn_older_hemi_ASEG_df,
                          ppmi_younger_hemi_ASEG_df, ppmi_older_hemi_ASEG_df], axis=0)
hemi_ASEG_df["ds_group"] = hemi_ASEG_df["ds"] + "-" + hemi_ASEG_df["group"]
hemi_ASEG_df["ds_hemi"] = hemi_ASEG_df["ds"] + "\n" + hemi_ASEG_df["hemi"]
print(f"hemi_ASEG_df shape: {hemi_ASEG_df.shape}")

hemi_ASEG_df = hemi_ASEG_df[hemi_ASEG_df["group"]!="prodromal"] .copy()

## Remove outliers
## This is structure specific (need to be QCed visually)
hemi_roi_list = ['Pallidum', 'Thalamus-Proper', 'Putamen',  'Amygdala', 'Caudate', 'Hippocampus', 'Accumbens-area', 
                 'Lateral-Ventricle']
min_vol_thresh_list = [1000,5000,2000,750,1500,2000,200,1000]
max_vol_thresh_list = [3000,10000,7000,2500,5500,6000,1000,60000]
outlier_min_thesh_dict = dict(zip(hemi_roi_list, min_vol_thresh_list))
outlier_max_thesh_dict = dict(zip(hemi_roi_list, max_vol_thresh_list))

print("Removing outliers")
for roi, thresh in outlier_min_thesh_dict.items():
    n_participants = hemi_ASEG_df["bids_id"].nunique()
    print(f"roi: {roi}, n_participants: {n_participants}")
    hemi_ASEG_df = hemi_ASEG_df[hemi_ASEG_df[roi] > thresh].copy()
    n_participants = hemi_ASEG_df["bids_id"].nunique()
    print(f"n_participants after outlier removal: {n_participants}")

for roi, thresh in outlier_max_thesh_dict.items():
    n_participants = hemi_ASEG_df["bids_id"].nunique()
    print(f"roi: {roi}, n_participants: {n_participants}")
    hemi_ASEG_df = hemi_ASEG_df[hemi_ASEG_df[roi] < thresh].copy()
    n_participants = hemi_ASEG_df["bids_id"].nunique()
    print(f"n_participants after outlier removal: {n_participants}")


hemi_ASEG_df.head()

In [ ]:
save_fig = True

# hemi_ASEG_df = hemi_ASEG_df[hemi_ASEG_df["dataset"].isin(["PPMI", "QPN", "NIMHANS_SERB"])]
hemi_ASEG_df = hemi_ASEG_df[hemi_ASEG_df["group"].isin(["control", "PD"])]

vol_ASEG_df_melt = hemi_ASEG_df.melt(
    id_vars=demo_cols + ["ds_group","ds_hemi"],
    var_name="ROI", 
    value_name="volume",
)

plot_df = vol_ASEG_df_melt.copy()
plot_df = plot_df.drop(columns=["participant_id"])

plot_df["ROI"] = plot_df["ROI"].astype(str)
hemi_roi_list = ['Pallidum', 'Thalamus-Proper', 'Putamen',  'Amygdala', 'Caudate', 'Hippocampus', 'Accumbens-area', 
                 'Lateral-Ventricle']
#'Cerebellum-Cortex','Cerebellum-White-Matter','VentralDC','Inf-Lat-Vent'

color_list = [  my_colors.NIM_SERB_CONTROL.value, my_colors.NIM_SERB_PD.value,
                my_colors.NIM_METAL_CONTROL.value, my_colors.NIM_METAL_PD.value,
                "white",
                my_colors.QPN_CONTROL.value, my_colors.QPN_PD.value, 
                my_colors.QPN_older_CONTROL.value, my_colors.QPN_older_PD.value,
                "white",
                my_colors.PPMI_CONTROL.value, my_colors.PPMI_PD.value,
                my_colors.PPMI_older_CONTROL.value, my_colors.PPMI_older_PD.value              
              ]

palette = sns.color_palette(palette=color_list)

hue_order = ["NIMHANS-1-control", "NIMHANS-1-PD", "NIMHANS-2-control", "NIMHANS-2-PD", "",
             "QPN-young-control", "QPN-young-PD", "QPN-older-control", "QPN-older-PD", "",
             "PPMI-young-control",  "PPMI-young-PD","PPMI-older-control",  "PPMI-older-PD"] 

sns.set_theme(font_scale=4)
with sns.axes_style("whitegrid"):
    g = sns.catplot(y="volume",x="hemi", hue="ds_group", col="ROI",kind="box", col_wrap=4, col_order=hemi_roi_list, 
                    hue_order=hue_order, palette=palette, data=plot_df, aspect=1, height=10, sharey=False, legend=False)
    # g.tick_params(axis='x', rotation=90, labelsize=14)

if save_fig:
    g.savefig(f"{figs_dir}/ASEG_hemi.png")

### Global volumes

In [ ]:
# NIMHANS (SERB + METAL)
nimhans_serb_global_ASEG_df = pd.read_csv(f"{nimhans_serb_agg_data_dir}{global_vol_csv}").drop(columns=["Unnamed: 0"])
nimhans_serb_global_ASEG_df["ds"] = "NIMHANS-1"
n_nimhans_serb_participants = len(nimhans_serb_global_ASEG_df["participant_id"].unique())

nimhans_metal_global_ASEG_df = pd.read_csv(f"{nimhans_metal_agg_data_dir}{global_vol_csv}").drop(columns=["Unnamed: 0"])
nimhans_metal_global_ASEG_df["ds"] = "NIMHANS-2"
n_nimhans_metal_participants = len(nimhans_metal_global_ASEG_df["participant_id"].unique())

# QPN
qpn_global_ASEG_df = pd.read_csv(f"{qpn_agg_data_dir}{global_vol_csv}").drop(columns=["Unnamed: 0"])
## Reanme eTIV col
qpn_global_ASEG_df = qpn_global_ASEG_df.rename(columns={"EstimatedTotalIntraCranialVol":"EstimatedTotalIntraCranial"})
qpn_global_ASEG_df["ds"] = "QPN"
n_qpn_participants = len(qpn_global_ASEG_df["participant_id"].unique())

# PPMI
ppmi_global_ASEG_df = pd.read_csv(f"{ppmi_agg_data_dir}{global_vol_csv}").drop(columns=["Unnamed: 0"])
ppmi_global_ASEG_df["ds"] = "PPMI"
n_ppmi_participants = len(ppmi_global_ASEG_df["participant_id"].unique())

print(f"n_nimhans_participants: {(n_nimhans_serb_participants, n_nimhans_metal_participants)}, n_qpn_participants:{n_qpn_participants}")


if match_age:
    print(f"Matching age < {age_thresh}")

    ppmi_younger_global_ASEG_df = ppmi_global_ASEG_df[ppmi_global_ASEG_df["age"] < age_thresh].copy()
    ppmi_younger_global_ASEG_df["ds"] = "PPMI-young"
    n_younger_ppmi_participants = len(ppmi_younger_global_ASEG_df["participant_id"].unique())

    ppmi_older_global_ASEG_df = ppmi_global_ASEG_df[ppmi_global_ASEG_df["age"] >= age_thresh].copy()
    ppmi_older_global_ASEG_df["ds"] = "PPMI-older"
    n_older_ppmi_participants = len(ppmi_older_global_ASEG_df["participant_id"].unique())

    qpn_younger_global_ASEG_df = qpn_global_ASEG_df[qpn_global_ASEG_df["age"] < age_thresh].copy()
    qpn_younger_global_ASEG_df["ds"] = "QPN-young"
    n_qpn_younger_participants = len(qpn_younger_global_ASEG_df["participant_id"].unique())

    qpn_older_global_ASEG_df = qpn_global_ASEG_df[qpn_global_ASEG_df["age"] >= age_thresh].copy()
    qpn_older_global_ASEG_df["ds"] = "QPN-older"
    n_qpn_older_participants = len(qpn_older_global_ASEG_df["participant_id"].unique())

    print(f"n_nimhans_participants: {(n_nimhans_serb_participants,n_nimhans_metal_participants)}")
    print(f"n_qpn_younger_participants:{n_qpn_younger_participants}, n_qpn_older_participants: {n_qpn_older_participants}")
    print(f"n_ppmi_younger_participants:{n_younger_ppmi_participants}, n_ppmi_older_participants: {n_older_ppmi_participants}")

# Concat
global_vol_ASEG_df = pd.concat([nimhans_serb_global_ASEG_df, nimhans_metal_global_ASEG_df, 
                          qpn_younger_global_ASEG_df, qpn_older_global_ASEG_df,
                          ppmi_younger_global_ASEG_df, ppmi_older_global_ASEG_df], axis=0)

global_vol_ASEG_df["ds_group"] = global_vol_ASEG_df["ds"] + "-" + global_vol_ASEG_df["group"]

global_vol_ASEG_df = global_vol_ASEG_df[global_vol_ASEG_df["group"]!="prodromal"] .copy()

print(f"global_vol_ASEG_df shape: {global_vol_ASEG_df.shape}")

## tmp
# qpn_global_ASEG_df = qpn_global_ASEG_df.drop(columns=["participant_id"])

qpn_global_ASEG_df.head()

In [ ]:
save_fig = False

# global_vol_ASEG_df = global_vol_ASEG_df[global_vol_ASEG_df["dataset"].isin(["PPMI", "QPN","NIMHANS_SERB"])]

global_vol_ASEG_df = global_vol_ASEG_df[global_vol_ASEG_df["group"].isin(["control", "PD"])]

global_vol_ASEG_df = global_vol_ASEG_df.rename(columns={"EstimatedTotalIntraCranial":"eTIV"})

global_vol_ASEG_df_melt = global_vol_ASEG_df.melt(
    id_vars=demo_cols + ["ds_group"],
    var_name="ROI", 
    value_name="volume",
)

plot_df = global_vol_ASEG_df_melt.copy()

global_roi_list = ["eTIV", "SupraTentorial", "TotalGray", "SubCortGray", 
                    "CSF","Brain-Stem","3rd-Ventricle","4th-Ventricle"]
plot_df = plot_df[plot_df["ROI"].isin(global_roi_list)]

color_list = [  my_colors.NIM_SERB_CONTROL.value, my_colors.NIM_SERB_PD.value,
                my_colors.NIM_METAL_CONTROL.value, my_colors.NIM_METAL_PD.value,
                "white",
                my_colors.QPN_CONTROL.value, my_colors.QPN_PD.value, 
                my_colors.QPN_older_CONTROL.value, my_colors.QPN_older_PD.value,
                "white",
                my_colors.PPMI_CONTROL.value, my_colors.PPMI_PD.value,
                my_colors.PPMI_older_CONTROL.value, my_colors.PPMI_older_PD.value              
              ]

palette = sns.color_palette(palette=color_list)

hue_order = ["NIMHANS-1-control", "NIMHANS-1-PD", "NIMHANS-2-control", "NIMHANS-2-PD", "",
             "QPN-young-control", "QPN-young-PD", "QPN-older-control", "QPN-older-PD", "",
             "PPMI-young-control",  "PPMI-young-PD","PPMI-older-control",  "PPMI-older-PD"] 

sns.set_theme(font_scale=4)
with sns.axes_style("whitegrid"):
    g = sns.catplot(y="volume",x="hemi", hue="ds_group", col="ROI", kind="box", col_wrap=4, 
                    col_order=global_roi_list, hue_order=hue_order,
    palette=palette, data=plot_df, aspect=1, height=10, sharey=False, legend=False)
    
    plt.legend(bbox_to_anchor =(0,2.5), loc='lower center',ncol=3, fontsize=40)

if save_fig:
    g.savefig(f"{figs_dir}/ASEG_global.png")

### Demographics

In [ ]:
demo_df = global_vol_ASEG_df[demo_cols].copy()
demo_df.groupby(["ds","group"]).count()

In [ ]:
demo_df.groupby(["ds","group"])["age"].describe()

### OLS

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
stat_df = global_vol_ASEG_df.copy()
dataset_list = ["NIMHANS-1","NIMHANS-2"]
hemi = "global"
stat_df = stat_df[(stat_df["ds"].isin(dataset_list)) & (stat_df["hemi"]==hemi)]

brain_var = "eTIV"
demo_vars = ["participant_id","age","sex","group","hemi","ds"]

input_var = demo_vars + [brain_var]

stat_df = stat_df[input_var]
stat_df.head()

In [ ]:
def get_stats(stat_df, ind_var, dep_var, cat_vars, global_correction=False):
    cvar_str = ""
    for cvar in cat_vars:
        cvar_str = "".join([f"{cvar_str} + C({cvar})"])    

    if global_correction:
        formula = f"{dep_var} ~ {ind_var}{cvar_str} + {global_correction}"
    else:
        formula = f"{dep_var} ~ {ind_var}{cvar_str}"
        
    print(f"formula: {formula}")
    res = smf.ols(formula=formula, data=stat_df).fit()
    
    tval_df = pd.DataFrame(columns=["tvalues"], data=res.tvalues)
    pval_df = pd.DataFrame(columns=["pvalues"], data=res.pvalues)
    res_df = pd.concat([tval_df, pval_df], axis=1)
    res_df = res_df.reset_index().rename(columns={"index":"var"})

    return res_df, res, formula

### Replication analysis
- Group differences within each dataset

In [ ]:
ind_var = "age"
cat_vars = ["sex", "group"]
dataset_list = ["NIMHANS-1","NIMHANS-2","QPN-young","PPMI-young","QPN-older","PPMI-older"]

pval_thresh = 0.05

### Global volumes

In [ ]:
brain_roi_list = ["eTIV", "SupraTentorial", "TotalGray", "SubCortGray", "CSF"]
hemi = "global"
res_df = pd.DataFrame()
for ds in dataset_list:
    for dep_var in brain_roi_list:
        print(f"ds:{ds}, ind_var:{ind_var}")
        stat_df = global_vol_ASEG_df.copy()
        stat_df = stat_df[(stat_df["ds"]==ds) & (stat_df["hemi"]==hemi)]
        _df, res, formula = get_stats(stat_df, ind_var, dep_var, cat_vars, global_correction=False)
        _df["ROI"] = dep_var
        _df["hemi"] = hemi
        _df["ds"] = ds
        res_df = pd.concat([res_df, _df], axis=0)


res_df[(res_df["var"]=="C(group)[T.control]") & (res_df["pvalues"] < pval_thresh)]

### Subcortical volumes

In [ ]:
brain_roi_list = ['Pallidum', 'ThalamusProper', 'Putamen',  'Amygdala', 'Caudate', 'Hippocampus', 'AccumbensArea']
hemi_list = ["lh", "rh"]
global_correction = "eTIV"
# rename columns with "-" to remove error in statsmodels
hemi_ASEG_df = hemi_ASEG_df.rename(columns={"Thalamus-Proper":"ThalamusProper", 'Accumbens-area':'AccumbensArea'})

res_df = pd.DataFrame()
for ds in dataset_list:
    for dep_var in brain_roi_list:
        for hemi in hemi_list:
            # print(f"ds:{ds}, ind_var:{ind_var}")
            stat_df = hemi_ASEG_df.copy()
            stat_df = stat_df[(stat_df["ds"]==ds) & (stat_df["hemi"]==hemi)].copy()

            if global_correction:
                stat_df = pd.merge(stat_df, global_vol_ASEG_df[["participant_id","eTIV"]], on="participant_id", how="left")
                _df, _, formula = get_stats(stat_df, ind_var, dep_var, cat_vars, global_correction)
            else:
                _df, _, formula = get_stats(stat_df, ind_var, dep_var, cat_vars)

            _df["ROI"] = dep_var
            _df["hemi"] = hemi
            _df["dataset"] = ds
            res_df = pd.concat([res_df, _df], axis=0)


res_df[(res_df["var"]=="C(group)[T.control]") & (res_df["pvalues"] < pval_thresh)]

### Compare older and younger groups

In [ ]:
global_correction = "eTIV"
ind_var = "age"
cat_vars = ["sex", "ds", "group"]

res_df = pd.DataFrame()

for dep_var in brain_roi_list:
    for hemi in hemi_list:
        # print(f"ds:{ds}, ind_var:{ind_var}")
        stat_df = hemi_ASEG_df.copy()
        stat_df = stat_df[stat_df["ds"].isin(["PPMI-young","PPMI-older"])].copy()
        stat_df = stat_df[(stat_df["hemi"]==hemi)].copy()

        if global_correction:
            stat_df = pd.merge(stat_df, global_vol_ASEG_df[["participant_id","eTIV"]], on="participant_id", how="left")
            _df, _, formula = get_stats(stat_df, ind_var, dep_var, cat_vars, global_correction)
        else:
            _df, _, formula = get_stats(stat_df, ind_var, dep_var, cat_vars)

        _df["ROI"] = dep_var
        _df["hemi"] = hemi
        # _df["dataset"] = ds
        res_df = pd.concat([res_df, _df], axis=0)

### CTh

In [ ]:
brain_roi_list = CT_DKT_df[CT_DKT_df.columns[~CT_DKT_df.columns.isin(demo_cols + ["ds_group","ds_hemi"])]].columns
print(f"n_brain_rois: {len(brain_roi_list)}")
hemi_list = ["lh", "rh"]

res_df = pd.DataFrame()
for ds in dataset_list:
    for dep_var in brain_roi_list:
        for hemi in hemi_list:
            # print(f"ds:{ds}, ind_var:{ind_var}")
            stat_df = CT_DKT_df.copy()
            stat_df = stat_df[(stat_df["ds"]==ds) & (stat_df["hemi"]==hemi)]
            _df, _, formula = get_stats(stat_df, ind_var, dep_var, cat_vars)
            _df["ROI"] = dep_var
            _df["hemi"] = hemi
            _df["ds"] = ds
            res_df = pd.concat([res_df, _df], axis=0)


res_df[(res_df["var"]=="C(group)[T.control]") & (res_df["pvalues"] < pval_thresh)]

In [ ]:
res_df[(res_df["var"]=="age") & (res_df["pvalues"] < pval_thresh)]

### Cross-cohort comparisons

In [ ]:
brain_roi_list = ["eTIV", "SupraTentorial", "TotalGray", "SubCortGray", "CSF"]
ind_var = "age"
cat_vars = ["sex", "ds_group"]
pval_thresh = 0.05

group_list = ["control","PD"]
hemi = "global"
res_df = pd.DataFrame()
for grp in group_list:
    for dep_var in brain_roi_list:
        # print(f"ds:{ds}, ind_var:{ind_var}")
        stat_df = global_vol_ASEG_df.copy()
        stat_df = stat_df[(stat_df["group"]==grp) & (stat_df["hemi"]==hemi)]
        _df, _, formula = get_stats(stat_df, ind_var, dep_var, cat_vars)
        _df["ROI"] = dep_var
        _df["hemi"] = hemi
        _df["group"] = grp
        res_df = pd.concat([res_df, _df], axis=0)


res_df[(~res_df["var"].isin(["Intercept","C(sex)[T.M]","age"])) & (res_df["pvalues"] < pval_thresh)]